
# Youtube - Scaper



In [1]:
from pyppeteer import launch
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import time

# for extended documentation visit --> https://miyakogi.github.io/pyppeteer/
# !!! function could only be called with await !!!
async def scrape(url_: str, selector_: str, page_function_ = "(element) => element.outerHTML",
                 bypass_google_anti_scrape_algorithm_ = False, log_ = True):
    if log_ : print("-------------------------Scrape Log Begin--------------------------", "\n")
    #create random user agent so YouTube's algorithm gets pypassed
    ua = UserAgent()
    agent = ua.random
    
    # create browser, incognito context and page
    browser = await launch()
    context = await browser.createIncognitoBrowserContext()
    page = await context.newPage()
    if log_ : print("Browser, Incognito Context and Page created")
    
    # set user agent
    await page.setUserAgent(agent)
    if log_ : print("User Agent:", agent)
    
    # open url
    await page.goto(url_)
    if log_ : print("Url opened:", url_)
        
    if bypass_google_anti_scrape_algorithm_:
        await page.waitForSelector("h1.title")
        await page.click("h1.title")
        time.sleep(5)
        await page.keyboard.press("End")
    
    # wait until page gets loaded
    await page.waitForSelector(selector_)
    if log_ : print("Selector loaded:", selector_)
        
    await page.click(selector_)
        
    if bypass_google_anti_scrape_algorithm_:
        time.sleep(3)
        
    await page.click(selector_)
    
    # get element from query selector and relating function
    request_result = await page.querySelectorEval(selector_, page_function_)
    if log_ : print("Request finished")

    # close browser
    await browser.close()
    if log_ : print("Browser closed", "\n")
    if log_ : print("-------------------------Scrape Log End----------------------------", "\n")
    
    return request_result

In [2]:
# get YouTube Video Title

url = "https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo"
query_selector = "h1.title"
function = "(element) => element.firstChild.innerHTML"

title = await scrape(url, query_selector, function)
                      
print(title)

-------------------------Scrape Log Begin-------------------------- 

Browser, Incognito Context and Page created
User Agent: Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1
Url opened: https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo
Selector loaded: h1.title
Request finished
Browser closed 

-------------------------Scrape Log End---------------------------- 

Sentiment Analysis Python - 1 -  Introduction to Emotion Analysis  (NLP)


In [3]:
# get comments and their authors as html

url = "https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo"
query_selector = "ytd-comments"
function = "(element) => element.outerHTML"

html = await scrape(url, query_selector, function, True)

-------------------------Scrape Log Begin-------------------------- 

Browser, Incognito Context and Page created
User Agent: Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/31.0.1650.16 Safari/537.36
Url opened: https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo
Selector loaded: ytd-comments
Request finished
Browser closed 

-------------------------Scrape Log End---------------------------- 



In [4]:
# parse html and assign them

def _parse_comments_with_corresponding_authors(html_, log_ = True):
    soup = BeautifulSoup(html, features="html.parser")

    # get authors of comments and clear html data
    authors = [item.text.strip() for item in soup.select("a[id=author-text] > span")]

    # get comments and clear html data
    comments = [item.text.strip().replace("\n", " ") for item in soup.select("yt-formatted-string[id=content-text]")]

    comments_with_authors = list(zip(authors, comments))
    
    if log_:
        print("Finished parsing")
        #for author, comment in comments_with_authors:
        #    print(author, "wrote:\n -" + comment)
    
    return comments_with_authors

_parse_comments_with_corresponding_authors(html, False)

[('morthim', "i'm really impressed by your emotions dict."),
 ('reda bekka', 'PLease can u make a course of mouvment detections'),
 ('IndieSpaceAstronaut', 'Love the hashtags lol'),
 ('Usama Iftikhar Butt', 'please upload 2 3 videos weekly'),
 ('Dr Scary', 'Our Boiii Is Back Lol :)'),
 ('vaishnavi kulkarni',
  'your space invader game tutorial is amazing! Hope to see more videos on python during Quarantine!!'),
 ('Alma Jose',
  'can someone please give me code for finding different emotions like sad happy angry and neutral from lyrics of song.'),
 ('Usama Iftikhar Butt', 'great series'),
 ('Hasna Bouazza', 'could we have the code .py'),
 ('Aditya Pai Thon', 'Boii is bacccc')]

In [5]:
# scrape and parse YouTube Title

# returns title as string
# function is asynchronous and therefore it has to be awaited
async def _scrape_and_parse_youtube_title(url: str, log_ = True):
    if "youtube.com" in url:
        return await scrape(url, "h1.title", "(element) => element.firstChild.innerHTML", log_ = log_)
    else:
        print("Wrong url format given!")
        
await _scrape_and_parse_youtube_title("https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo", log_ = False)

'Sentiment Analysis Python - 1 -  Introduction to Emotion Analysis  (NLP)'

In [6]:
# scrape and parse comments with authors

# returns list of tuples [(Author, Comment), (...), ...]
# function is asynchronous and therefore it has to be awaited
async def _scrape_and_parse_youtube_comments(url: str, log_ = True):
    if "youtube.com" in url:
        html = await scrape(url, "ytd-comments", "(element) => element.outerHTML", bypass_google_anti_scrape_algorithm_ = True, log_ = log_)
        
        return _parse_comments_with_corresponding_authors(html, log_ = log_)
    else:
        print("Wrong url format given!")
        
await _scrape_and_parse_youtube_comments("https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo", log_ = False)

[('morthim', "i'm really impressed by your emotions dict."),
 ('reda bekka', 'PLease can u make a course of mouvment detections'),
 ('IndieSpaceAstronaut', 'Love the hashtags lol'),
 ('Usama Iftikhar Butt', 'please upload 2 3 videos weekly'),
 ('Dr Scary', 'Our Boiii Is Back Lol :)'),
 ('vaishnavi kulkarni',
  'your space invader game tutorial is amazing! Hope to see more videos on python during Quarantine!!'),
 ('Alma Jose',
  'can someone please give me code for finding different emotions like sad happy angry and neutral from lyrics of song.'),
 ('Usama Iftikhar Butt', 'great series'),
 ('Hasna Bouazza', 'could we have the code .py'),
 ('Aditya Pai Thon', 'Boii is bacccc')]

In [7]:

# Scraping transaction : 

def _scrape_and_store_video(tx, httpUrl_, title_): # "tx" is a neo4j transaction...
    result = tx.run("CREATE (v:Video) "
                    "SET v = {title: $title, url: $url}"
                    "RETURN v.title + ', from node ' + id(v)", title=video_title, url=httpUrl_)
    
    return result

def _scrape_and_store_video_comments(tx, httpUrl_, comments_with_authors_):
    author_result = []
    comment_result = []
    for author, comment in comments_with_authors_:
        author_result.append(tx.run("CREATE (a:Author)"
                  "SET a = {name: $name}"
                  "RETURN a.name + ', created as Author with id ' + id(a)", name=author))
        
        comment_result.append(tx.run(
            """
                MATCH (v:Video), (a:Author)
                WHERE v.url = '%s' AND a.name = '%s'
                CREATE (a) - [r:%s { text: "%s" }] -> (v)
                RETURN v.title, type(r), r.text, a.name
            """ % ( httpUrl_, author, "COMMENTED", comment)))

    return zip(author_result, comment_result)

## Using Neo4j for data storage : ##

In [8]:
from neo4j import GraphDatabase

uri, user, password = 'bolt://localhost:7687', 'neo4j', 'neo4j_'


In [10]:
httpUrls = [
    "https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo"
]


driver = GraphDatabase.driver(uri, auth=(user, password))

# resetting database

with driver.session() as session:
    def _q(query) : return session.run(query)
    #---------------------------------------

    _q("MATCH (n) DETACH DELETE n") # remove all graphs and nodes! BE CAREFUL!

    #---------------------------------------
driver.close()

with driver.session() as session:
    for url in httpUrls :
        # run await outside of transaction because asynchronous transactions for Neo4j are not yet available for Python
        comments_with_authors = await _scrape_and_parse_youtube_comments(url, log_ = True)
        video_title = await _scrape_and_parse_youtube_title(url, log_ = True)
        
        result = session.write_transaction(_scrape_and_store_video, url, video_title)
        #print(result)
        
        result = session.write_transaction(_scrape_and_store_video_comments, url, comments_with_authors)
        #print(result)

driver.close()

-------------------------Scrape Log Begin-------------------------- 

Browser, Incognito Context and Page created
User Agent: Mozilla/5.0 (Windows; U; MSIE 9.0; WIndows NT 9.0; en-US))
Url opened: https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo
Selector loaded: ytd-comments
Request finished
Browser closed 

-------------------------Scrape Log End---------------------------- 

Finished parsing
-------------------------Scrape Log Begin-------------------------- 

Browser, Incognito Context and Page created
User Agent: Mozilla/5.0 (Windows; U; Windows NT 6.1; ko-KR) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27
Url opened: https://www.youtube.com/watch?v=dyN_WtjdfpA&list=PLhTjy8cBISEoOtB5_nwykvB9wfEDscuEo
Selector loaded: h1.title
Request finished
Browser closed 

-------------------------Scrape Log End---------------------------- 

